ЛАБОРАТОРНАЯ РАБОТА 5.1


In [ ]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys

# согласованная матричная норма
def matrix_norm(matrix):
  sum = 0
  for i in range(len(matrix)):
    sum += abs(matrix[i])
  return max(sum)

# согласованная векторная норма
def vec_norm(v):
  return max(map(abs, v))

# генерация вектора
def generate_vec(l, r, n):
    while True:
      vec = np.random.uniform(l, r, n)
      if vec_norm(vec) < 1:
        break
    return vec

# генерация матрицы
def generate_matrix(l, r, n):
    a = np.random.uniform(l, r, (n, n))
    return a

def increase_diag_elems(a, diag):
    n = len(a)
    for i in range(0, len(a)):
        a[i][i] = diag * sum(abs(a[i][j]) if j != i else 0 for j in range(n))
    return a

# Вычисляем диагональное преобладание
# Проверить выполнение условия диагонального преобладания
def calc_diagonal_dominance(a):
  degree = max(abs(a[i][i]) - sum(abs(a[i][j]) if j != i else 0 for j in range(len(a))) for i in range(len(a)))
  return degree > 0


In [ ]:
# стандартный метод Гаусса
def gauss(matrix, vec):
    n = len(matrix)
    A = deepcopy(matrix)
    b = deepcopy(vec)
    x = np.zeros(shape=(n, ))
    # towards
    for i in range(n - 1):
        if A[i][i] == 0:
            for j in range(i + 1, n):
                if A[j][i] != 0:
                    A[i], A[j] = A[j], A[i]
                    break

        for j in range(i + 1, n):
            c = - A[j][i] / A[i][i]
            A[j] += c * A[i]
            b[j] += c * b[i]

    # backwards
    for i in range(n - 1, -1, -1):
        x[i] = b[i] / A[i][i]
        for j in range(i - 1, -1, -1):
            b[j] -= A[j][i] * x[i]

    return np.array(x)

метод Якоби

In [ ]:
def jacobi(A, f):
  eps = eps = 1e-6
  n = len(A)
  D = np.array([[0 if i != j else A[i][j] for j in range(n)] for i in range(n)])
  D_inv = np.linalg.inv(D)
  P = np.dot(-D_inv, A - D)

  norm_P = matrix_norm(deepcopy(P))
  q = np.max(np.abs(np.linalg.eigvals(P)))
  if not (norm_P < 1):
    print('||P|| ≤ q < 1 not working')
    print(f'norma_P: {norm_P}')
  else:
    print('||P|| ≤ q < 1 correct')
    print(f'norma_P: {norm_P}')

  g = np.dot(D_inv, f)
  x_k = g
  iters = 1
  while True:
    x_ki = np.dot(P, x_k) + g
    # критерий остановки итерационного процесс используя равномерную норму
    if vec_norm(x_ki - x_k) < eps:
      break
    iters += 1
    x_k = x_ki
  return x_ki, iters


тестирование

In [ ]:
n = 10
a = generate_matrix(0, 10, n)
a = increase_diag_elems(a, 3)
print('matrix a:')
print(a)
f = generate_vec(0, 0.5, n)
print(f'\nvector f: {f}')

diag_cond = calc_diagonal_dominance(a)
print(f'\ncheck for diagonal dominance condition: {diag_cond}')

# correct res
x = np.dot(np.linalg.inv(a), f)

# jacobi res
x_j, iters = jacobi(a, f)

print(f'jacobi iterations: {iters}')
# gauss res
x_g = gauss(a, f)

print(f'\ncorrect res: {x}')
print(f'\njacobi res: {x_j}')
print(f'\ngauss res: {x_g}')

print('\n\nrelative errors:')
from tabulate import tabulate
g = vec_norm(x_g - x) / vec_norm(x_g)
j = vec_norm(x_j - x) / vec_norm(x_j)
mydata = [[g, j]]

# create header
head = ["Jacobi", "Gauss"]

# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

matrix a:
[[144.97561466   9.16381466   9.25421316   6.21727221   5.57038817
    8.07527481   2.59785917   1.88257564   3.75902006   1.80478701]
 [  5.39570649 115.22388431   2.39397764   9.14577052   2.71531561
    0.86663494   2.73075289   6.05541836   7.8783051    1.22607989]
 [  6.43069531   3.10528649 101.49360379   0.47120823   2.86130315
    1.95455403   7.91862718   2.69908653   6.20846721   2.18197314]
 [  7.56262108   6.44263741   3.61166505 171.34205967   2.34143891
    8.61527615   7.2185058    7.86560749   6.0716131    7.38465489]
 [  5.20924534   5.56477755   3.03002561   1.78521576 135.7496333
    5.34928475   1.16241279   8.76045733   5.64235047   8.74610816]
 [  3.83321992   3.25366125   9.10035653   1.27739288   8.55390764
  149.88976614   4.18063237   4.67936269   9.19982741   5.88489468]
 [  6.22738329   5.71663493   1.49596839   1.44603041   9.67829332
    2.06802323 131.42636921   6.29471654   2.05879028   8.82294933]
 [  8.34591933   3.19210406   3.74815506   4.4